<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES
================================================================================
Trabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion
en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Diciembre 2025

DESCRIPCION:
Fase 2B del analisis comparativo de modelos de segmentacion. Fusiona las metricas
de segmentacion calculadas en Fase 2A con las caracteristicas fotograficas
extraidas en Fase 1, permitiendo analizar como los parametros de captura
(apertura, ISO, exposicion) y las propiedades de imagen (contraste, nitidez,
complejidad textural) afectan al rendimiento de cada modelo.

DEPENDENCIAS DE DATOS:
- Requiere Fase 2A completada (CSVs de metricas por modelo)
- Requiere JSONs de caracteristicas fotograficas (20 archivos)
- Genera dataset fusionado para fases posteriores

ANALISIS INCLUIDOS:
1. Consolidacion de CSVs de metricas de todos los modelos
2. Extraccion de caracteristicas fotograficas clave (20 variables)
3. Fusion de datasets por codigo de fotografia
4. Correlaciones globales: caracteristicas vs IoU
5. Correlaciones por modelo: sensibilidad diferencial
6. Test de hipotesis especificas (bokeh, contraste, exposicion, fondo)
7. Tamano de efecto entre modelos (Cohen's d, eta cuadrado)
8. Estadisticas descriptivas y resumen ejecutivo

CARACTERISTICAS FOTOGRAFICAS EXTRAIDAS:
- EXIF: apertura, ISO, exposicion, focal
- Calidad: brillo, subexposicion, sobreexposicion, rango dinamico
- Contraste: RMS, nitidez Laplacian, SNR
- Textura: homogeneidad GLCM, entropia Haralick, contraste Haralick
- Frecuencia: ratio alta frecuencia
- Saliencia: centro, ratio centro/periferia, distancia centroide
- Color: entropia, saturacion

VERSION ACTUALIZADA:
- Extrae 152 caracteristicas fotograficas (vs 20 en version anterior)
- Incluye todas las texturas: Haralick, GLCM, LBP, Zernike
- Incluye analisis de frecuencia completo
- Incluye bordes y gradientes
- Correlaciones con multiples metricas objetivo (IoU, Dice, Boundary IoU)

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2b_correlaciones/
├── metricas_fusionadas.csv           # Dataset completo (67 + 152 columnas)
├── caracteristicas_fotograficas.csv  # 152 caracteristicas por foto
├── correlaciones_globales.csv        # Todas las correlaciones
├── correlaciones_por_modelo.csv      # Correlaciones segmentadas
├── correlaciones_multimetrica.csv    # Correlaciones vs IoU, Dice, Boundary
├── test_hipotesis.json               # Resultados de tests
├── tamano_efecto.json                # Cohen's d entre modelos
├── estadisticas_descriptivas.json    # Stats completas

================================================================================
"""

"\n================================================================================\nFASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES\n================================================================================\nTrabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion\nen Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universidad Oberta de Cataluna (UOC)\nFecha: Diciembre 2025\n\nDESCRIPCION:\nFase 2B del analisis comparativo de modelos de segmentacion. Fusiona las metricas\nde segmentacion calculadas en Fase 2A con las caracteristicas fotograficas\nextraidas en Fase 1, permitiendo analizar como los parametros de captura\n(apertura, ISO, exposicion) y las propiedades de imagen (contraste, nitidez,\ncomplejidad textural) afectan al rendimiento de cada modelo.\n\nDEPENDENCIAS DE DATOS:\n- Requiere Fase 2A completada (CSVs de metricas por modelo)\n- Requiere JSONs de caracteristicas fotograficas (20 archivos)\n- Genera dataset fusionado para fases poster

In [19]:
# =============================================================================
# IMPORTS
# =============================================================================

import json
import logging
import sys
import warnings
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy import stats

warnings.filterwarnings('ignore')



In [20]:
# =============================================================================
# FUNCIONES DE UTILIDAD PARA SERIALIZACION
# =============================================================================

def convertir_a_serializable(obj: Any) -> Any:
    """
    Convierte tipos numpy y pandas a tipos Python nativos para JSON.

    Args:
        obj: Objeto a convertir

    Returns:
        Objeto con tipos serializables
    """
    if isinstance(obj, dict):
        return {k: convertir_a_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convertir_a_serializable(v) for v in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        if np.isnan(obj) or np.isinf(obj):
            return None
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    return obj

In [21]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ConfiguracionFase2B:
    """
    Configuracion central para el pipeline de Fase 2B.

    Attributes:
        ruta_base_tfm: Ruta raiz del proyecto TFM
        ruta_csvs_fase2a: Directorio con CSVs de metricas de Fase 2A
        ruta_jsons_caracteristicas: Directorio con JSONs de caracteristicas
        ruta_salida: Directorio para resultados de Fase 2B
        nivel_significancia: Alpha para tests estadisticos (default 0.05)
        min_observaciones_correlacion: Minimo de observaciones para calcular correlacion
    """
    ruta_base_tfm: Path
    ruta_csvs_fase2a: Path = None
    ruta_jsons_caracteristicas: Path = None
    ruta_salida: Path = None
    nivel_significancia: float = 0.05
    min_observaciones_correlacion: int = 10

    def __post_init__(self):
        if self.ruta_csvs_fase2a is None:
            self.ruta_csvs_fase2a = self.ruta_base_tfm / "3_Analisis" / "fase2_evaluacion" / "metricas_agregadas"
        if self.ruta_jsons_caracteristicas is None:
            self.ruta_jsons_caracteristicas = self.ruta_base_tfm / "1_Caracteristicas" / "json"
        if self.ruta_salida is None:
            self.ruta_salida = self.ruta_base_tfm / "3_Analisis" / "fase2b_correlaciones"

In [22]:
# =============================================================================
# CONSTANTES: CARACTERISTICAS FOTOGRAFICAS A EXTRAER
# =============================================================================

CARACTERISTICAS_EXHAUSTIVAS = {
    # =========================================================================
    # METADATOS DE ARCHIVO (9 campos)
    # =========================================================================
    'meta_ancho': ('metadatos_archivo', 'ancho_original'),
    'meta_alto': ('metadatos_archivo', 'alto_original'),
    'meta_tamano_mb': ('metadatos_archivo', 'tamano_archivo_mb'),

    # =========================================================================
    # EXIF - PARAMETROS DE CAPTURA (14 campos)
    # =========================================================================
    'exif_apertura': ('metadatos_exif', 'apertura_fnumber'),
    'exif_iso': ('metadatos_exif', 'iso'),
    'exif_exposicion_seg': ('metadatos_exif', 'tiempo_exposicion_segundos'),
    'exif_focal': ('metadatos_exif', 'distancia_focal'),

    # =========================================================================
    # ESTADISTICAS DE COLOR - RGB (12 campos)
    # =========================================================================
    'color_rgb_r_mean': ('estadisticas_color', 'rgb', 'mean', 0),
    'color_rgb_g_mean': ('estadisticas_color', 'rgb', 'mean', 1),
    'color_rgb_b_mean': ('estadisticas_color', 'rgb', 'mean', 2),
    'color_rgb_r_std': ('estadisticas_color', 'rgb', 'stddev', 0),
    'color_rgb_g_std': ('estadisticas_color', 'rgb', 'stddev', 1),
    'color_rgb_b_std': ('estadisticas_color', 'rgb', 'stddev', 2),

    # =========================================================================
    # ESTADISTICAS DE COLOR - HSV (6 campos)
    # =========================================================================
    'color_hsv_hue_mean': ('estadisticas_color', 'hsv', 'hue_mean'),
    'color_hsv_hue_std': ('estadisticas_color', 'hsv', 'hue_std'),
    'color_hsv_sat_mean': ('estadisticas_color', 'hsv', 'saturation_mean'),
    'color_hsv_sat_std': ('estadisticas_color', 'hsv', 'saturation_std'),
    'color_hsv_val_mean': ('estadisticas_color', 'hsv', 'value_mean'),
    'color_hsv_val_std': ('estadisticas_color', 'hsv', 'value_std'),

    # =========================================================================
    # ESTADISTICAS DE COLOR - LAB (6 campos)
    # =========================================================================
    'color_lab_l_mean': ('estadisticas_color', 'lab', 'l_mean'),
    'color_lab_l_std': ('estadisticas_color', 'lab', 'l_std'),
    'color_lab_a_mean': ('estadisticas_color', 'lab', 'a_mean'),
    'color_lab_a_std': ('estadisticas_color', 'lab', 'a_std'),
    'color_lab_b_mean': ('estadisticas_color', 'lab', 'b_mean'),
    'color_lab_b_std': ('estadisticas_color', 'lab', 'b_std'),

    # =========================================================================
    # ESTADISTICAS DE COLOR - GLOBAL (3 campos)
    # =========================================================================
    'color_brillo': ('estadisticas_color', 'global', 'brillo_promedio'),
    'color_contraste': ('estadisticas_color', 'global', 'contraste'),
    'color_entropia': ('estadisticas_color', 'global', 'entropia'),

    # =========================================================================
    # HISTOGRAMAS (12 campos - estadisticas por canal)
    # =========================================================================
    'hist_rojo_pico': ('histogramas', 'rojo', 'pico_principal'),
    'hist_rojo_media': ('histogramas', 'rojo', 'media'),
    'hist_rojo_std': ('histogramas', 'rojo', 'std'),
    'hist_verde_pico': ('histogramas', 'verde', 'pico_principal'),
    'hist_verde_media': ('histogramas', 'verde', 'media'),
    'hist_verde_std': ('histogramas', 'verde', 'std'),
    'hist_azul_pico': ('histogramas', 'azul', 'pico_principal'),
    'hist_azul_media': ('histogramas', 'azul', 'media'),
    'hist_azul_std': ('histogramas', 'azul', 'std'),
    'hist_intensidad_entropia': ('histogramas', 'intensidad', 'entropia'),

    # =========================================================================
    # TEXTURAS - HARALICK (13 campos)
    # =========================================================================
    'tex_haralick_asm': ('texturas', 'haralick', 'angular_second_moment'),
    'tex_haralick_contrast': ('texturas', 'haralick', 'contrast'),
    'tex_haralick_correlation': ('texturas', 'haralick', 'correlation'),
    'tex_haralick_sum_squares': ('texturas', 'haralick', 'sum_of_squares'),
    'tex_haralick_idm': ('texturas', 'haralick', 'inverse_diff_moment'),
    'tex_haralick_sum_avg': ('texturas', 'haralick', 'sum_average'),
    'tex_haralick_sum_var': ('texturas', 'haralick', 'sum_variance'),
    'tex_haralick_sum_entropy': ('texturas', 'haralick', 'sum_entropy'),
    'tex_haralick_entropy': ('texturas', 'haralick', 'entropy'),
    'tex_haralick_diff_var': ('texturas', 'haralick', 'difference_variance'),
    'tex_haralick_diff_entropy': ('texturas', 'haralick', 'difference_entropy'),
    'tex_haralick_imc1': ('texturas', 'haralick', 'info_measure_correlation_1'),
    'tex_haralick_imc2': ('texturas', 'haralick', 'info_measure_correlation_2'),

    # =========================================================================
    # TEXTURAS - GLCM (12 campos)
    # =========================================================================
    'tex_glcm_contrast_mean': ('texturas', 'glcm', 'contrast_mean'),
    'tex_glcm_contrast_std': ('texturas', 'glcm', 'contrast_std'),
    'tex_glcm_dissimilarity_mean': ('texturas', 'glcm', 'dissimilarity_mean'),
    'tex_glcm_dissimilarity_std': ('texturas', 'glcm', 'dissimilarity_std'),
    'tex_glcm_homogeneity_mean': ('texturas', 'glcm', 'homogeneity_mean'),
    'tex_glcm_homogeneity_std': ('texturas', 'glcm', 'homogeneity_std'),
    'tex_glcm_energy_mean': ('texturas', 'glcm', 'energy_mean'),
    'tex_glcm_energy_std': ('texturas', 'glcm', 'energy_std'),
    'tex_glcm_correlation_mean': ('texturas', 'glcm', 'correlation_mean'),
    'tex_glcm_correlation_std': ('texturas', 'glcm', 'correlation_std'),
    'tex_glcm_asm_mean': ('texturas', 'glcm', 'ASM_mean'),
    'tex_glcm_asm_std': ('texturas', 'glcm', 'ASM_std'),

    # =========================================================================
    # TEXTURAS - LBP (3 campos)
    # =========================================================================
    'tex_lbp_mean': ('texturas', 'lbp', 'mean'),
    'tex_lbp_std': ('texturas', 'lbp', 'std'),
    'tex_lbp_entropy': ('texturas', 'lbp', 'entropy'),

    # =========================================================================
    # BORDES - CANNY (3 campos)
    # =========================================================================
    'borde_canny_sigma1': ('bordes_caracteristicas', 'canny', 'densidad_sigma_1.0'),
    'borde_canny_sigma2': ('bordes_caracteristicas', 'canny', 'densidad_sigma_2.0'),
    'borde_canny_sigma3': ('bordes_caracteristicas', 'canny', 'densidad_sigma_3.0'),

    # =========================================================================
    # BORDES - ESQUINAS (3 campos)
    # =========================================================================
    'borde_harris_count': ('bordes_caracteristicas', 'esquinas', 'harris'),
    'borde_shi_tomasi_count': ('bordes_caracteristicas', 'esquinas', 'shi_tomasi'),
    'borde_harris_densidad': ('bordes_caracteristicas', 'esquinas', 'densidad_harris'),

    # =========================================================================
    # BORDES - GRADIENTES (5 campos)
    # =========================================================================
    'borde_sobel_mean': ('bordes_caracteristicas', 'gradientes', 'sobel_mean'),
    'borde_sobel_max': ('bordes_caracteristicas', 'gradientes', 'sobel_max'),
    'borde_sobel_std': ('bordes_caracteristicas', 'gradientes', 'sobel_std'),
    'borde_scharr_mean': ('bordes_caracteristicas', 'gradientes', 'scharr_mean'),
    'borde_prewitt_mean': ('bordes_caracteristicas', 'gradientes', 'prewitt_mean'),

    # =========================================================================
    # BORDES - HOG (3 campos)
    # =========================================================================
    'borde_hog_mean': ('bordes_caracteristicas', 'hog', 'mean'),
    'borde_hog_std': ('bordes_caracteristicas', 'hog', 'std'),
    'borde_hog_max': ('bordes_caracteristicas', 'hog', 'max'),

    # =========================================================================
    # CALIDAD - NITIDEZ (3 campos)
    # =========================================================================
    'calidad_nitidez_laplacian': ('calidad', 'nitidez', 'laplacian'),
    'calidad_nitidez_tenengrad': ('calidad', 'nitidez', 'tenengrad'),
    'calidad_nitidez_varianza': ('calidad', 'nitidez', 'varianza_normalizada'),

    # =========================================================================
    # CALIDAD - EXPOSICION (5 campos)
    # =========================================================================
    'calidad_brillo_medio': ('calidad', 'exposicion', 'brillo_medio'),
    'calidad_sobreexp_pct': ('calidad', 'exposicion', 'sobre_expuesto_pct'),
    'calidad_subexp_pct': ('calidad', 'exposicion', 'sub_expuesto_pct'),
    'calidad_exp_entropia': ('calidad', 'exposicion', 'entropia'),
    'calidad_rango_dinamico': ('calidad', 'exposicion', 'rango_dinamico'),

    # =========================================================================
    # CALIDAD - CONTRASTE (2 campos)
    # =========================================================================
    'calidad_contraste_rms': ('calidad', 'contraste', 'rms'),
    'calidad_contraste_michelson': ('calidad', 'contraste', 'michelson'),

    # =========================================================================
    # CALIDAD - BALANCE BLANCOS (4 campos)
    # =========================================================================
    'calidad_wb_red': ('calidad', 'balance_blancos', 'mean_red'),
    'calidad_wb_green': ('calidad', 'balance_blancos', 'mean_green'),
    'calidad_wb_blue': ('calidad', 'balance_blancos', 'mean_blue'),
    'calidad_wb_desviacion': ('calidad', 'balance_blancos', 'desviacion_canales'),

    # =========================================================================
    # CALIDAD - RUIDO (4 campos)
    # =========================================================================
    'calidad_ruido_laplacian': ('calidad', 'ruido', 'laplacian'),
    'calidad_ruido_median': ('calidad', 'ruido', 'median_filter'),
    'calidad_snr': ('calidad', 'ruido', 'snr'),
    'calidad_snr_db': ('calidad', 'ruido', 'snr_db'),

    # =========================================================================
    # ANALISIS DE FRECUENCIA (8 campos)
    # =========================================================================
    'freq_energia_baja': ('analisis_frecuencia', 'energia_frecuencia_baja'),
    'freq_energia_media': ('analisis_frecuencia', 'energia_frecuencia_media'),
    'freq_energia_alta': ('analisis_frecuencia', 'energia_frecuencia_alta'),
    'freq_ratio_baja': ('analisis_frecuencia', 'ratio_baja'),
    'freq_ratio_media': ('analisis_frecuencia', 'ratio_media'),
    'freq_ratio_alta': ('analisis_frecuencia', 'ratio_alta'),
    'freq_entropia_espectral': ('analisis_frecuencia', 'entropia_espectral'),
    'freq_pico_dominante': ('analisis_frecuencia', 'pico_dominante'),

    # =========================================================================
    # SALIENCIA VISUAL - ESPECTRAL (5 campos)
    # =========================================================================
    'sal_espectral_media': ('saliencia_visual', 'espectral', 'media'),
    'sal_espectral_std': ('saliencia_visual', 'espectral', 'std'),
    'sal_espectral_max': ('saliencia_visual', 'espectral', 'max'),
    'sal_espectral_p90': ('saliencia_visual', 'espectral', 'percentil_90'),
    'sal_espectral_p95': ('saliencia_visual', 'espectral', 'percentil_95'),

    # =========================================================================
    # SALIENCIA VISUAL - DISTRIBUCION ESPACIAL (8 campos)
    # =========================================================================
    'sal_superior_izq': ('saliencia_visual', 'distribucion_espacial', 'saliencia_superior_izquierda'),
    'sal_superior_der': ('saliencia_visual', 'distribucion_espacial', 'saliencia_superior_derecha'),
    'sal_inferior_izq': ('saliencia_visual', 'distribucion_espacial', 'saliencia_inferior_izquierda'),
    'sal_inferior_der': ('saliencia_visual', 'distribucion_espacial', 'saliencia_inferior_derecha'),
    'sal_variabilidad_cuadrantes': ('saliencia_visual', 'distribucion_espacial', 'variabilidad_cuadrantes'),
    'sal_centro': ('saliencia_visual', 'distribucion_espacial', 'saliencia_centro'),
    'sal_periferia': ('saliencia_visual', 'distribucion_espacial', 'saliencia_periferia'),
    'sal_ratio_centro_periferia': ('saliencia_visual', 'distribucion_espacial', 'ratio_centro_periferia'),

    # =========================================================================
    # SALIENCIA VISUAL - CENTROIDE (4 campos)
    # =========================================================================
    'sal_centroide_x': ('saliencia_visual', 'centroide', 'centroide_x_normalizado'),
    'sal_centroide_y': ('saliencia_visual', 'centroide', 'centroide_y_normalizado'),
    'sal_distancia_centro': ('saliencia_visual', 'centroide', 'distancia_desde_centro'),
    'sal_area_saliente_pct': ('saliencia_visual', 'centroide', 'area_saliente_porcentaje'),
}

# Zernike moments (25 campos) - se extraen por separado
ZERNIKE_MOMENTS = 25

In [23]:
# =============================================================================
# CLASE: EXTRACTOR DE CARACTERISTICAS
# =============================================================================

class ExtractorCaracteristicas:
    """
    Extrae caracteristicas exhaustivas de JSONs fotograficos.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def _extraer_valor_anidado(self, data: Dict, keys: Tuple) -> Any:
        """Extrae valor de diccionario anidado siguiendo ruta de claves."""
        resultado = data
        for key in keys:
            if isinstance(key, int):
                # Indice de lista
                if isinstance(resultado, list) and len(resultado) > key:
                    resultado = resultado[key]
                else:
                    return None
            elif isinstance(resultado, dict) and key in resultado:
                resultado = resultado[key]
            else:
                return None
        return resultado

    def extraer_caracteristicas_json(self, ruta_json: Path) -> Dict[str, Any]:
        """
        Extrae todas las caracteristicas de un JSON de fotografia.

        Args:
            ruta_json: Ruta al archivo JSON

        Returns:
            Diccionario con ~120 caracteristicas
        """
        with open(ruta_json, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Obtener nombre de foto
        nombre = data.get('metadatos_archivo', {}).get('nombre_archivo', '')
        nombre = nombre.replace('.jpg', '').replace('.JPG', '')

        caracteristicas = {'codigo_foto': nombre}

        # Extraer caracteristicas definidas
        for nombre_col, ruta_keys in CARACTERISTICAS_EXHAUSTIVAS.items():
            valor = self._extraer_valor_anidado(data, ruta_keys)

            # Convertir a numerico si es string numerico
            if valor is not None and isinstance(valor, str):
                try:
                    valor = float(valor)
                except (ValueError, TypeError):
                    valor = None

            caracteristicas[nombre_col] = valor

        # Extraer Zernike moments
        zernike = data.get('texturas', {}).get('zernike', [])
        for i in range(ZERNIKE_MOMENTS):
            if i < len(zernike):
                caracteristicas[f'tex_zernike_{i:02d}'] = zernike[i]
            else:
                caracteristicas[f'tex_zernike_{i:02d}'] = None

        return caracteristicas

    def cargar_todas_caracteristicas(self, rutas_json: List[Path]) -> pd.DataFrame:
        """
        Carga caracteristicas de todos los JSONs.

        Args:
            rutas_json: Lista de rutas a archivos JSON

        Returns:
            DataFrame con caracteristicas por fotografia
        """
        self.logger.info(f"Extrayendo caracteristicas de {len(rutas_json)} fotografias...")

        caracteristicas_list = []

        for ruta in rutas_json:
            try:
                caract = self.extraer_caracteristicas_json(ruta)
                caracteristicas_list.append(caract)
            except Exception as e:
                self.logger.error(f"  Error en {ruta.name}: {e}")

        df = pd.DataFrame(caracteristicas_list)

        # Contar campos extraidos
        cols_con_datos = df.notna().sum()
        cols_completas = (cols_con_datos == len(df)).sum()

        self.logger.info(f"  Fotografias procesadas: {len(df)}")
        self.logger.info(f"  Caracteristicas totales: {len(df.columns) - 1}")  # -1 por codigo_foto
        self.logger.info(f"  Caracteristicas completas (sin NaN): {cols_completas}")

        return df

In [24]:
# =============================================================================
# CLASE: CARGADOR DE METRICAS
# =============================================================================

class CargadorMetricas:
    """Carga y consolida CSVs de metricas de Fase 2A."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def cargar_csvs(self, rutas_csv: Dict[str, Path]) -> pd.DataFrame:
        """
        Consolida multiples CSVs en un DataFrame.

        Args:
            rutas_csv: Diccionario {nombre_modelo: ruta_csv}

        Returns:
            DataFrame consolidado
        """
        self.logger.info("Cargando CSVs de metricas de Fase 2A...")

        dataframes = []

        for modelo, ruta in rutas_csv.items():
            if not ruta.exists():
                self.logger.warning(f"  {modelo}: NO ENCONTRADO - {ruta}")
                continue

            try:
                df = pd.read_csv(ruta)

                # Asegurar columna modelo
                if 'modelo' not in df.columns:
                    df['modelo'] = modelo

                self.logger.info(f"  {modelo}: {len(df)} filas, {len(df.columns)} cols")
                dataframes.append(df)

            except Exception as e:
                self.logger.error(f"  Error cargando {modelo}: {e}")

        if not dataframes:
            raise ValueError("No se pudieron cargar CSVs")

        df_consolidado = pd.concat(dataframes, ignore_index=True)
        self.logger.info(f"Total consolidado: {len(df_consolidado)} filas")

        return df_consolidado

In [25]:
# =============================================================================
# CLASE: ANALIZADOR DE CORRELACIONES
# =============================================================================

class AnalizadorCorrelaciones:
    """Analiza correlaciones entre caracteristicas y metricas."""

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def calcular_correlaciones_globales(self, df: pd.DataFrame,
                                        metricas_objetivo: List[str] = None) -> pd.DataFrame:
        """
        Calcula correlaciones Pearson y Spearman entre caracteristicas y metricas.

        Args:
            df: DataFrame fusionado
            metricas_objetivo: Lista de metricas objetivo (default: iou, dice, boundary_iou)

        Returns:
            DataFrame con correlaciones
        """
        if metricas_objetivo is None:
            metricas_objetivo = ['iou', 'dice', 'boundary_iou']

        self.logger.info(f"Calculando correlaciones globales vs {metricas_objetivo}...")

        # Identificar columnas de caracteristicas
        cols_excluir = ['codigo_foto', 'config_codigo', 'modelo'] + metricas_objetivo
        cols_excluir += [c for c in df.columns if c.startswith('haralick_')]  # Haralick de mascara
        cols_excluir += ['precision', 'recall', 'f1_score']  # Otras metricas

        cols_foto = [c for c in df.columns
                    if c not in cols_excluir
                    and df[c].dtype in ['float64', 'int64', 'float32', 'int32']
                    and not c.startswith('bbox_')
                    and not c.startswith('centroide_')
                    and c not in ['area_px', 'perimetro_px']]

        correlaciones = []

        for col in cols_foto:
            for metrica in metricas_objetivo:
                if metrica not in df.columns:
                    continue

                mask = df[col].notna() & df[metrica].notna()
                n_obs = mask.sum()

                if n_obs >= self.config.min_observaciones_correlacion:
                    try:
                        # Pearson
                        r_pearson, p_pearson = stats.pearsonr(
                            df.loc[mask, col], df.loc[mask, metrica]
                        )

                        # Spearman
                        r_spearman, p_spearman = stats.spearmanr(
                            df.loc[mask, col], df.loc[mask, metrica]
                        )

                        correlaciones.append({
                            'caracteristica': col,
                            'metrica_objetivo': metrica,
                            'r_pearson': r_pearson,
                            'p_pearson': p_pearson,
                            'r_spearman': r_spearman,
                            'p_spearman': p_spearman,
                            'n_observaciones': n_obs,
                            'sig_pearson': p_pearson < self.config.nivel_significancia,
                            'sig_spearman': p_spearman < self.config.nivel_significancia
                        })
                    except Exception:
                        pass

        df_corr = pd.DataFrame(correlaciones)

        if len(df_corr) > 0:
            df_corr = df_corr.sort_values('r_pearson', key=abs, ascending=False)

            n_sig = df_corr['sig_pearson'].sum()
            self.logger.info(f"  Correlaciones calculadas: {len(df_corr)}")
            self.logger.info(f"  Significativas (p<0.05): {n_sig}")

        return df_corr

    def calcular_correlaciones_por_modelo(self, df: pd.DataFrame,
                                          metrica_objetivo: str = 'iou') -> pd.DataFrame:
        """
        Calcula correlaciones segmentadas por modelo.

        Args:
            df: DataFrame fusionado
            metrica_objetivo: Metrica objetivo

        Returns:
            DataFrame con correlaciones por modelo
        """
        self.logger.info(f"Calculando correlaciones por modelo vs {metrica_objetivo}...")

        cols_foto = [c for c in df.columns if c not in
                    ['codigo_foto', 'config_codigo', 'modelo', metrica_objetivo]
                    and df[c].dtype in ['float64', 'int64', 'float32', 'int32']]

        resultados = []

        for modelo in df['modelo'].unique():
            df_m = df[df['modelo'] == modelo]

            for col in cols_foto:
                mask = df_m[col].notna() & df_m[metrica_objetivo].notna()
                n_obs = mask.sum()

                if n_obs >= self.config.min_observaciones_correlacion:
                    try:
                        r, p = stats.pearsonr(
                            df_m.loc[mask, col], df_m.loc[mask, metrica_objetivo]
                        )

                        resultados.append({
                            'modelo': modelo,
                            'caracteristica': col,
                            'correlacion': r,
                            'p_valor': p,
                            'n_observaciones': n_obs,
                            'significativo': p < self.config.nivel_significancia
                        })
                    except Exception:
                        pass

        return pd.DataFrame(resultados)

In [26]:
# =============================================================================
# CLASE: ANALIZADOR DE HIPOTESIS
# =============================================================================

class AnalizadorHipotesis:
    """Ejecuta tests de hipotesis especificas."""

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def _calcular_cohens_d(self, g1: pd.Series, g2: pd.Series) -> float:
        """Calcula Cohen's d entre dos grupos."""
        n1, n2 = len(g1), len(g2)
        var1, var2 = g1.var(), g2.var()
        pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
        if pooled_std == 0:
            return 0.0
        return (g1.mean() - g2.mean()) / pooled_std

    def _interpretar_d(self, d: float) -> str:
        """Interpreta Cohen's d."""
        d_abs = abs(d)
        if d_abs < 0.2: return "insignificante"
        elif d_abs < 0.5: return "pequeno"
        elif d_abs < 0.8: return "mediano"
        return "grande"

    def test_apertura_bokeh(self, df: pd.DataFrame) -> Dict:
        """H1: Apertura abierta (bokeh) mejora segmentacion."""
        self.logger.info("Testing H1: Efecto de apertura (bokeh)...")

        col_apertura = 'exif_apertura'
        if col_apertura not in df.columns:
            return {'error': 'Columna apertura no disponible'}

        mask = df[col_apertura].notna() & (df[col_apertura] > 0)
        df_h = df[mask].copy()

        umbral = 2.8
        grupo_bokeh = df_h[df_h[col_apertura] < umbral]['iou']
        grupo_nitido = df_h[df_h[col_apertura] >= umbral]['iou']

        resultado = {
            'hipotesis': 'H1: Apertura abierta (bokeh) mejora segmentacion',
            'variable': col_apertura,
            'umbral': umbral,
            'grupo_bokeh': {
                'n': len(grupo_bokeh),
                'iou_mean': float(grupo_bokeh.mean()) if len(grupo_bokeh) > 0 else None,
                'iou_std': float(grupo_bokeh.std()) if len(grupo_bokeh) > 0 else None
            },
            'grupo_nitido': {
                'n': len(grupo_nitido),
                'iou_mean': float(grupo_nitido.mean()) if len(grupo_nitido) > 0 else None,
                'iou_std': float(grupo_nitido.std()) if len(grupo_nitido) > 0 else None
            }
        }

        if len(grupo_bokeh) > 1 and len(grupo_nitido) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_bokeh, grupo_nitido)
            d = self._calcular_cohens_d(grupo_bokeh, grupo_nitido)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def test_complejidad_fondo(self, df: pd.DataFrame) -> Dict:
        """H2: Fondos complejos (alta entropia textural) dificultan segmentacion."""
        self.logger.info("Testing H2: Efecto de complejidad del fondo...")

        col_entropia = 'tex_haralick_entropy'
        if col_entropia not in df.columns:
            return {'error': 'Columna entropia no disponible'}

        mask = df[col_entropia].notna()
        df_h = df[mask].copy()

        mediana = df_h[col_entropia].median()
        grupo_simple = df_h[df_h[col_entropia] <= mediana]['iou']
        grupo_complejo = df_h[df_h[col_entropia] > mediana]['iou']

        resultado = {
            'hipotesis': 'H2: Fondos complejos dificultan segmentacion',
            'variable': col_entropia,
            'umbral_mediana': float(mediana),
            'grupo_simple': {
                'n': len(grupo_simple),
                'iou_mean': float(grupo_simple.mean()) if len(grupo_simple) > 0 else None
            },
            'grupo_complejo': {
                'n': len(grupo_complejo),
                'iou_mean': float(grupo_complejo.mean()) if len(grupo_complejo) > 0 else None
            }
        }

        if len(grupo_simple) > 1 and len(grupo_complejo) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_simple, grupo_complejo)
            d = self._calcular_cohens_d(grupo_simple, grupo_complejo)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def test_contraste(self, df: pd.DataFrame) -> Dict:
        """H3: Mayor contraste mejora segmentacion."""
        self.logger.info("Testing H3: Efecto del contraste...")

        col_contraste = 'calidad_contraste_rms'
        if col_contraste not in df.columns:
            return {'error': 'Columna contraste no disponible'}

        mask = df[col_contraste].notna()
        df_h = df[mask].copy()

        mediana = df_h[col_contraste].median()
        grupo_bajo = df_h[df_h[col_contraste] <= mediana]['iou']
        grupo_alto = df_h[df_h[col_contraste] > mediana]['iou']

        resultado = {
            'hipotesis': 'H3: Mayor contraste mejora segmentacion',
            'variable': col_contraste,
            'umbral_mediana': float(mediana),
            'grupo_bajo': {
                'n': len(grupo_bajo),
                'iou_mean': float(grupo_bajo.mean()) if len(grupo_bajo) > 0 else None
            },
            'grupo_alto': {
                'n': len(grupo_alto),
                'iou_mean': float(grupo_alto.mean()) if len(grupo_alto) > 0 else None
            }
        }

        if len(grupo_bajo) > 1 and len(grupo_alto) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_bajo, grupo_alto)
            d = self._calcular_cohens_d(grupo_alto, grupo_bajo)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def test_saliencia_central(self, df: pd.DataFrame) -> Dict:
        """H4: Sujeto centrado (alta saliencia central) facilita deteccion."""
        self.logger.info("Testing H4: Efecto de saliencia central...")

        col_saliencia = 'sal_centro'
        if col_saliencia not in df.columns:
            return {'error': 'Columna saliencia no disponible'}

        mask = df[col_saliencia].notna()
        df_h = df[mask].copy()

        mediana = df_h[col_saliencia].median()
        grupo_periferico = df_h[df_h[col_saliencia] <= mediana]['iou']
        grupo_central = df_h[df_h[col_saliencia] > mediana]['iou']

        resultado = {
            'hipotesis': 'H4: Saliencia central facilita segmentacion',
            'variable': col_saliencia,
            'umbral_mediana': float(mediana),
            'grupo_periferico': {
                'n': len(grupo_periferico),
                'iou_mean': float(grupo_periferico.mean()) if len(grupo_periferico) > 0 else None
            },
            'grupo_central': {
                'n': len(grupo_central),
                'iou_mean': float(grupo_central.mean()) if len(grupo_central) > 0 else None
            }
        }

        if len(grupo_periferico) > 1 and len(grupo_central) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_periferico, grupo_central)
            d = self._calcular_cohens_d(grupo_central, grupo_periferico)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def test_nitidez(self, df: pd.DataFrame) -> Dict:
        """H5: Mayor nitidez mejora segmentacion."""
        self.logger.info("Testing H5: Efecto de nitidez...")

        col_nitidez = 'calidad_nitidez_laplacian'
        if col_nitidez not in df.columns:
            return {'error': 'Columna nitidez no disponible'}

        mask = df[col_nitidez].notna()
        df_h = df[mask].copy()

        mediana = df_h[col_nitidez].median()
        grupo_borroso = df_h[df_h[col_nitidez] <= mediana]['iou']
        grupo_nitido = df_h[df_h[col_nitidez] > mediana]['iou']

        resultado = {
            'hipotesis': 'H5: Mayor nitidez mejora segmentacion',
            'variable': col_nitidez,
            'umbral_mediana': float(mediana),
            'grupo_borroso': {
                'n': len(grupo_borroso),
                'iou_mean': float(grupo_borroso.mean()) if len(grupo_borroso) > 0 else None
            },
            'grupo_nitido': {
                'n': len(grupo_nitido),
                'iou_mean': float(grupo_nitido.mean()) if len(grupo_nitido) > 0 else None
            }
        }

        if len(grupo_borroso) > 1 and len(grupo_nitido) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_borroso, grupo_nitido)
            d = self._calcular_cohens_d(grupo_nitido, grupo_borroso)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def test_frecuencia_alta(self, df: pd.DataFrame) -> Dict:
        """H6: Alta frecuencia (detalles finos) afecta segmentacion."""
        self.logger.info("Testing H6: Efecto de frecuencia alta (detalles)...")

        col_freq = 'freq_ratio_alta'
        if col_freq not in df.columns:
            return {'error': 'Columna frecuencia no disponible'}

        mask = df[col_freq].notna()
        df_h = df[mask].copy()

        mediana = df_h[col_freq].median()
        grupo_bajo = df_h[df_h[col_freq] <= mediana]['iou']
        grupo_alto = df_h[df_h[col_freq] > mediana]['iou']

        resultado = {
            'hipotesis': 'H6: Alta frecuencia afecta segmentacion',
            'variable': col_freq,
            'umbral_mediana': float(mediana),
            'grupo_bajo_detalle': {
                'n': len(grupo_bajo),
                'iou_mean': float(grupo_bajo.mean()) if len(grupo_bajo) > 0 else None
            },
            'grupo_alto_detalle': {
                'n': len(grupo_alto),
                'iou_mean': float(grupo_alto.mean()) if len(grupo_alto) > 0 else None
            }
        }

        if len(grupo_bajo) > 1 and len(grupo_alto) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_bajo, grupo_alto)
            d = self._calcular_cohens_d(grupo_alto, grupo_bajo)

            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }
            resultado['cohens_d'] = float(d)
            resultado['interpretacion'] = self._interpretar_d(d)

        return resultado

    def ejecutar_todos(self, df: pd.DataFrame) -> Dict:
        """Ejecuta todos los tests de hipotesis."""
        return {
            'H1_apertura_bokeh': self.test_apertura_bokeh(df),
            'H2_complejidad_fondo': self.test_complejidad_fondo(df),
            'H3_contraste': self.test_contraste(df),
            'H4_saliencia_central': self.test_saliencia_central(df),
            'H5_nitidez': self.test_nitidez(df),
            'H6_frecuencia_alta': self.test_frecuencia_alta(df)
        }

In [27]:
# =============================================================================
# CLASE: ANALIZADOR DE EFECTO ENTRE MODELOS
# =============================================================================

class AnalizadorEfectoModelos:
    """Calcula tamanos de efecto entre modelos."""

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def calcular_cohens_d_pareado(self, df: pd.DataFrame,
                                   metrica: str = 'iou') -> pd.DataFrame:
        """Calcula Cohen's d para todos los pares de modelos."""
        self.logger.info("Calculando Cohen's d entre modelos...")

        modelos = sorted(df['modelo'].unique())
        comparaciones = []

        for i, m1 in enumerate(modelos):
            for m2 in modelos[i+1:]:
                g1 = df[df['modelo'] == m1][metrica].dropna()
                g2 = df[df['modelo'] == m2][metrica].dropna()

                if len(g1) > 1 and len(g2) > 1:
                    n1, n2 = len(g1), len(g2)
                    var1, var2 = g1.var(), g2.var()
                    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))

                    d = (g1.mean() - g2.mean()) / pooled_std if pooled_std > 0 else 0
                    t_stat, p_val = stats.ttest_ind(g1, g2)

                    comparaciones.append({
                        'modelo_1': m1,
                        'modelo_2': m2,
                        'mean_1': float(g1.mean()),
                        'mean_2': float(g2.mean()),
                        'n_1': n1,
                        'n_2': n2,
                        'cohens_d': float(d),
                        'interpretacion': self._interpretar_d(d),
                        't_statistic': float(t_stat),
                        'p_valor': float(p_val)
                    })

        return pd.DataFrame(comparaciones)

    def calcular_eta_cuadrado(self, df: pd.DataFrame, metrica: str = 'iou') -> Dict:
        """Calcula eta cuadrado (ANOVA)."""
        self.logger.info("Calculando eta cuadrado (ANOVA)...")

        grupos = [df[df['modelo'] == m][metrica].dropna().values
                  for m in df['modelo'].unique()]

        f_stat, p_val = stats.f_oneway(*grupos)

        grand_mean = df[metrica].mean()
        ss_total = ((df[metrica] - grand_mean) ** 2).sum()

        ss_between = 0
        for m in df['modelo'].unique():
            grupo = df[df['modelo'] == m][metrica]
            ss_between += len(grupo) * (grupo.mean() - grand_mean) ** 2

        eta_sq = ss_between / ss_total if ss_total > 0 else 0

        return {
            'metrica': metrica,
            'n_modelos': len(df['modelo'].unique()),
            'f_statistic': float(f_stat),
            'p_valor': float(p_val),
            'eta_squared': float(eta_sq),
            'interpretacion': self._interpretar_eta(eta_sq)
        }

    def _interpretar_d(self, d: float) -> str:
        d_abs = abs(d)
        if d_abs < 0.2: return "insignificante"
        elif d_abs < 0.5: return "pequeno"
        elif d_abs < 0.8: return "mediano"
        return "grande"

    def _interpretar_eta(self, eta: float) -> str:
        if eta < 0.01: return "insignificante"
        elif eta < 0.06: return "pequeno"
        elif eta < 0.14: return "mediano"
        return "grande"

In [28]:
# =============================================================================
# CLASE: GENERADOR DE ESTADISTICAS
# =============================================================================

class GeneradorEstadisticas:
    """Genera estadisticas descriptivas."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def estadisticas_por_modelo(self, df: pd.DataFrame) -> Dict:
        """Calcula estadisticas por modelo."""
        self.logger.info("Generando estadisticas por modelo...")

        metricas = ['iou', 'dice', 'precision', 'recall', 'boundary_iou']
        metricas_disponibles = [m for m in metricas if m in df.columns]

        stats_modelo = {}

        for modelo in df['modelo'].unique():
            df_m = df[df['modelo'] == modelo]

            stats_modelo[modelo] = {
                'n_evaluaciones': int(len(df_m)),
                'n_configuraciones': int(df_m['config_codigo'].nunique()) if 'config_codigo' in df_m.columns else 0,
                'n_fotos': int(df_m['codigo_foto'].nunique())
            }

            for metrica in metricas_disponibles:
                vals = df_m[metrica].dropna()
                if len(vals) > 0:
                    stats_modelo[modelo][metrica] = {
                        'mean': float(vals.mean()),
                        'std': float(vals.std()),
                        'min': float(vals.min()),
                        'max': float(vals.max()),
                        'median': float(vals.median()),
                        'q25': float(vals.quantile(0.25)),
                        'q75': float(vals.quantile(0.75))
                    }

        return stats_modelo

    def estadisticas_por_foto(self, df: pd.DataFrame) -> Dict:
        """Calcula estadisticas por fotografia."""
        self.logger.info("Generando estadisticas por fotografia...")

        stats_foto = {}

        for foto in df['codigo_foto'].unique():
            df_f = df[df['codigo_foto'] == foto]

            stats_foto[foto] = {
                'n_evaluaciones': int(len(df_f)),
                'iou_mean': float(df_f['iou'].mean()),
                'iou_std': float(df_f['iou'].std()),
                'iou_min': float(df_f['iou'].min()),
                'iou_max': float(df_f['iou'].max()),
                'mejor_modelo': df_f.loc[df_f['iou'].idxmax(), 'modelo'],
                'mejor_iou': float(df_f['iou'].max())
            }

        return stats_foto

    def ranking_global(self, df: pd.DataFrame, top_n: int = 20) -> pd.DataFrame:
        """Genera ranking global de configuraciones."""
        self.logger.info(f"Generando ranking TOP-{top_n}...")

        ranking = df.groupby(['modelo', 'config_codigo']).agg({
            'iou': ['mean', 'std', 'min', 'max', 'count']
        }).round(4)

        ranking.columns = ['iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n_fotos']
        ranking = ranking.reset_index()
        ranking = ranking.sort_values('iou_mean', ascending=False)

        return ranking.head(top_n)

In [29]:
# =============================================================================
# CLASE: ORQUESTADOR FASE 2B
# =============================================================================

class OrquestadorFase2B:
    """Orquesta la ejecucion completa de Fase 2B."""

    def __init__(self, config: ConfiguracionFase2B):
        self.config = config
        self.logger = self._configurar_logger()

        # Componentes
        self.extractor = ExtractorCaracteristicas(self.logger)
        self.cargador = CargadorMetricas(self.logger)
        self.analizador_corr = AnalizadorCorrelaciones(config, self.logger)
        self.analizador_hip = AnalizadorHipotesis(config, self.logger)
        self.analizador_efecto = AnalizadorEfectoModelos(config, self.logger)
        self.generador_stats = GeneradorEstadisticas(self.logger)

        # Resultados
        self.df_fusionado = None
        self.df_caracteristicas = None
        self.correlaciones_globales = None
        self.correlaciones_por_modelo = None
        self.resultados_hipotesis = None
        self.tamano_efecto = None
        self.estadisticas = None

    def _configurar_logger(self) -> logging.Logger:
        logger = logging.getLogger('Fase2B')
        logger.setLevel(logging.INFO)
        logger.handlers = []

        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(logging.INFO)
        formatter = logging.Formatter(
            '[%(asctime)s] %(levelname)-8s | %(message)s',
            datefmt='%H:%M:%S'
        )
        handler.setFormatter(formatter)
        logger.addHandler(handler)

        return logger

    def ejecutar(self, rutas_csv: Dict[str, Path], rutas_json: List[Path]) -> None:
        """Ejecuta el pipeline completo."""
        self.logger.info("=" * 70)
        self.logger.info("FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES")
        self.logger.info("(VERSION EXHAUSTIVA - 152 caracteristicas fotograficas)")
        self.logger.info("=" * 70)

        inicio = datetime.now()

        # Paso 1: Cargar metricas
        self.logger.info("\n[PASO 1/8] Cargando metricas de Fase 2A...")
        df_metricas = self.cargador.cargar_csvs(rutas_csv)

        # Paso 2: Extraer caracteristicas fotograficas
        self.logger.info("\n[PASO 2/8] Extrayendo caracteristicas fotograficas (152 campos)...")
        self.df_caracteristicas = self.extractor.cargar_todas_caracteristicas(rutas_json)

        # Paso 3: Fusionar
        self.logger.info("\n[PASO 3/8] Fusionando datasets...")
        self.df_fusionado = df_metricas.merge(
            self.df_caracteristicas,
            on='codigo_foto',
            how='left'
        )
        self.logger.info(f"  Dataset fusionado: {len(self.df_fusionado)} filas x {len(self.df_fusionado.columns)} columnas")

        # Paso 4: Correlaciones globales
        self.logger.info("\n[PASO 4/8] Calculando correlaciones globales...")
        self.correlaciones_globales = self.analizador_corr.calcular_correlaciones_globales(
            self.df_fusionado, ['iou', 'dice', 'boundary_iou']
        )

        # Paso 5: Correlaciones por modelo
        self.logger.info("\n[PASO 5/8] Calculando correlaciones por modelo...")
        self.correlaciones_por_modelo = self.analizador_corr.calcular_correlaciones_por_modelo(
            self.df_fusionado
        )

        # Paso 6: Tests de hipotesis
        self.logger.info("\n[PASO 6/8] Ejecutando tests de hipotesis...")
        self.resultados_hipotesis = self.analizador_hip.ejecutar_todos(self.df_fusionado)

        # Paso 7: Tamano de efecto
        self.logger.info("\n[PASO 7/8] Calculando tamano de efecto entre modelos...")
        self.tamano_efecto = {
            'cohens_d_pareado': self.analizador_efecto.calcular_cohens_d_pareado(
                self.df_fusionado
            ).to_dict('records'),
            'anova_eta_squared': self.analizador_efecto.calcular_eta_cuadrado(
                self.df_fusionado
            )
        }

        # Paso 8: Estadisticas
        self.logger.info("\n[PASO 8/8] Generando estadisticas y guardando...")
        self.estadisticas = {
            'por_modelo': self.generador_stats.estadisticas_por_modelo(self.df_fusionado),
            'por_foto': self.generador_stats.estadisticas_por_foto(self.df_fusionado),
            'ranking_top20': self.generador_stats.ranking_global(self.df_fusionado, 20).to_dict('records')
        }

        self._guardar_resultados()

        duracion = (datetime.now() - inicio).total_seconds()

        self.logger.info("\n" + "=" * 70)
        self.logger.info("FASE 2B COMPLETADA")
        self.logger.info("=" * 70)
        self.logger.info(f"Duracion: {duracion:.1f} segundos")
        self.logger.info(f"Evaluaciones: {len(self.df_fusionado)}")
        self.logger.info(f"Caracteristicas fotograficas: {len(self.df_caracteristicas.columns) - 1}")
        self.logger.info(f"Correlaciones calculadas: {len(self.correlaciones_globales)}")

    def _guardar_resultados(self) -> None:
        """Guarda todos los resultados."""
        self.config.ruta_salida.mkdir(parents=True, exist_ok=True)

        # CSV fusionado
        ruta = self.config.ruta_salida / 'metricas_fusionadas.csv'
        self.df_fusionado.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Caracteristicas
        ruta = self.config.ruta_salida / 'caracteristicas_fotograficas.csv'
        self.df_caracteristicas.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Correlaciones globales
        ruta = self.config.ruta_salida / 'correlaciones_globales.csv'
        self.correlaciones_globales.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Correlaciones por modelo
        ruta = self.config.ruta_salida / 'correlaciones_por_modelo.csv'
        self.correlaciones_por_modelo.to_csv(ruta, index=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Tests hipotesis
        ruta = self.config.ruta_salida / 'test_hipotesis.json'
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.resultados_hipotesis), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Tamano efecto
        ruta = self.config.ruta_salida / 'tamano_efecto.json'
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.tamano_efecto), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta}")

        # Estadisticas
        ruta = self.config.ruta_salida / 'estadisticas_descriptivas.json'
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.estadisticas), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta}")

    def imprimir_resumen(self) -> None:
        """Imprime resumen de resultados."""
        print("\n" + "=" * 70)
        print("RESUMEN DE RESULTADOS FASE 2B")
        print("=" * 70)

        # Top correlaciones
        print("\nTOP 10 CORRELACIONES CON IoU:")
        print("-" * 50)
        df_iou = self.correlaciones_globales[
            self.correlaciones_globales['metrica_objetivo'] == 'iou'
        ].head(10)

        for _, row in df_iou.iterrows():
            sig = "*" if row['sig_pearson'] else ""
            print(f"  {row['caracteristica']:<35} r={row['r_pearson']:>7.4f} {sig}")

        # Por modelo
        print("\nRENDIMIENTO POR MODELO:")
        print("-" * 50)
        for modelo, stats in sorted(
            self.estadisticas['por_modelo'].items(),
            key=lambda x: x[1].get('iou', {}).get('mean', 0),
            reverse=True
        ):
            if 'iou' in stats:
                iou = stats['iou']
                print(f"  {modelo:<15} IoU={iou['mean']:.4f} +/- {iou['std']:.4f}")

        # ANOVA
        anova = self.tamano_efecto['anova_eta_squared']
        print(f"\nANOVA - Efecto del modelo:")
        print(f"  eta^2 = {anova['eta_squared']:.4f} ({anova['interpretacion']})")
        print(f"  El modelo explica {anova['eta_squared']*100:.1f}% de la varianza en IoU")

In [30]:
# =============================================================================
# FUNCION PRINCIPAL
# =============================================================================

def ejecutar_fase2b(ruta_base_tfm: str,
                    rutas_csv: Dict[str, str],
                    rutas_json: List[str]) -> OrquestadorFase2B:
    """
    Ejecuta el pipeline completo de Fase 2B.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM
        rutas_csv: Diccionario {modelo: ruta_csv}
        rutas_json: Lista de rutas a JSONs de caracteristicas

    Returns:
        OrquestadorFase2B con resultados
    """
    config = ConfiguracionFase2B(ruta_base_tfm=Path(ruta_base_tfm))

    orquestador = OrquestadorFase2B(config)

    rutas_csv_path = {k: Path(v) for k, v in rutas_csv.items()}
    rutas_json_path = [Path(r) for r in rutas_json]

    orquestador.ejecutar(rutas_csv_path, rutas_json_path)
    orquestador.imprimir_resumen()

    return orquestador

In [31]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":

    from google.colab import drive
    drive.mount('/content/drive')

    RUTA_BASE_TFM = Path("/content/drive/MyDrive/TFM")
    RUTA_FASE2A = RUTA_BASE_TFM / "3_Analisis" / "fase2_evaluacion" / "metricas_agregadas"
    RUTA_CARACTERISTICAS = RUTA_BASE_TFM / "1_Caracteristicas" / "json"

    # CSVs de metricas
    rutas_csv = {
        'yolov8': RUTA_FASE2A / "todas_metricas.csv",
        'bodypix': RUTA_FASE2A / "bodypix_todas_metricas.csv",
        'mask2former': RUTA_FASE2A / "m2f_todas_metricas.csv",
        'oneformer': RUTA_FASE2A / "of_todas_metricas.csv",
        'sam2': RUTA_FASE2A / "s2_todas_metricas.csv",
        'sam2_prompts': RUTA_FASE2A / "s2p_todas_metricas.csv"
    }

    # JSONs de caracteristicas
    rutas_json = sorted(RUTA_CARACTERISTICAS.glob("_DSC*_caracteristicas.json"))

    print("Verificando archivos...")
    for modelo, ruta in rutas_csv.items():
        existe = "OK" if ruta.exists() else "NO ENCONTRADO"
        print(f"  {modelo}: {existe}")
    print(f"  JSONs encontrados: {len(rutas_json)}")

    # Ejecutar
    orquestador = ejecutar_fase2b(
        ruta_base_tfm=str(RUTA_BASE_TFM),
        rutas_csv={k: str(v) for k, v in rutas_csv.items()},
        rutas_json=[str(r) for r in rutas_json]
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Verificando archivos...
  yolov8: OK
  bodypix: OK
  mask2former: OK
  oneformer: OK
  sam2: OK
  sam2_prompts: OK
  JSONs encontrados: 20
[22:58:27] INFO     | ======================================================================


INFO:Fase2B:======================================================================


[22:58:27] INFO     | FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES


INFO:Fase2B:FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES


[22:58:27] INFO     | (VERSION EXHAUSTIVA - 152 caracteristicas fotograficas)


...

[22:58:40] INFO     | Caracteristicas fotograficas: 148


INFO:Fase2B:Caracteristicas fotograficas: 148


[22:58:40] INFO     | Correlaciones calculadas: 528


INFO:Fase2B:Correlaciones calculadas: 528



RESUMEN DE RESULTADOS FASE 2B

TOP 10 CORRELACIONES CON IoU:
--------------------------------------------------
  chamfer_distance                    r=-0.7212 *
  hausdorff_distance                  r=-0.6464 *
  solidity                            r= 0.5575 *
  compacidad                          r= 0.5444 *
  circularity                         r= 0.5444 *
  ratio_solidity                      r= 0.5213 *
  rectangularity                      r= 0.4953 *
  ratio_componente_principal          r= 0.4897 *
  aspect_ratio                        r=-0.3943 *
  diferencia_compacidad               r=-0.3641 *

RENDIMIENTO POR MODELO:
--------------------------------------------------
  yolov8          IoU=0.9498 +/- 0.0527
  oneformer       IoU=0.8790 +/- 0.2185
  mask2former     IoU=0.6933 +/- 0.3572
  bodypix         IoU=0.6559 +/- 0.1740
  sam2            IoU=0.4169 +/- 0.3772

ANOVA - Efecto del modelo:
  eta^2 = 0.3797 (grande)
  El modelo explica 38.0% de la varianza en IoU
